In [1]:
#"D:\[TH]Học máy và ứng dụng\lab3\drug200.csv"
# nhập thư viện
import numpy as np
import pandas as pd
# tạo hàm lấy dữ liệu
def loadCsv(filename) -> pd.DataFrame:
    return pd.read_csv(filename)

# tạo hàm biến đổi cột định tính, dùng phương pháp one hot
def transform(data, columns_trans): # data dạng dataframe, data_trans là cột cần biến đổi --> dạng Series, nhiều cột cần biến đổi thì bỏ vào list
    for i in columns_trans:
        unique = data[i].unique() + '-' + i # trả lại mảng
        # tạo ma trận 0
        matrix_0 = np.zeros((len(data), len(unique)), dtype = int)
        frame_0 = pd.DataFrame(matrix_0, columns = unique)
        for index, value in enumerate(data[i]):
            frame_0.at[index, value + '-' + i] = 1
        data[unique] = frame_0
    return data # trả lại data truyền vào nhưng đã bị biến đổi
# tạo hàm scale dữ liệu về [0,1] (min max scaler)
def scale_data(data, columns_scale): # columns_scale là cột cần scale, nếu nhiều bỏ vào list ['a', 'b']
    for i in columns_scale:  
        _max = data[i].max()
        _min = data[i].min()
        data[i] = (data[i] - _min) / (_max - _min)
        min_max_scaller = lambda x: round( 3)
        data[i] = data[i].apply(min_max_scaller)
    return data # --> trả về frame
# hàm tính khoảng cách Cosine 
def cosine_distance(train_X, test_X): # cả 2 đều dạng mảng
    dict_distance = dict()
    for index, value in enumerate(test_X, start = 1):
        for j in train_X:
            result = np.sqrt(np.sum((j - value)**2))
            if index not in dict_distance:
                dict_distance[index] = [result]
            else:
                dict_distance[index].append(result)
    return dict_distance # {1: [6.0, 5.0], 2: [4.6, 3.1]}
# hàm gán kết quả theo k
def pred_test(k, train_X, test_X, train_y): # train_X, test_X là mảng, train_y là Series
    lst_predict = list()
    dict_distance = cosine_distance(train_X, test_X)
    train_y = train_y.to_frame(name = 'target').reset_index(drop = True) # train_y là frame
    frame_concat = pd.concat([pd.DataFrame(dict_distance), train_y], axis = 1)
    for i in range(1, len(dict_distance) + 1):
        sort_distance = frame_concat[[i, 'target']].sort_values(by = i, ascending = True)[:k] # sắp xếp và lấy k
        target_predict = sort_distance['target'].value_counts(ascending = False).index[0]
        lst_predict.append([i, target_predict])
    return lst_predict
## Demo qua drug200
file_path = r"D:\[TH]Học máy và ứng dụng\WEEK3_Hoàng Đặng Bảo Trân - 2274802011053\drug200.csv"
data = loadCsv('drug200.csv')
data.head()
df = transform(data, ['Sex', 'BP', 'Cholesterol']).drop(['Sex', 'BP', 'Cholesterol'], axis = 1)
df
scale_data(df, ['Age', 'Na_to_K']) # không cần gán biến vì nó trả về data đã truyền vào nhưng đã bị scale
df
# tạo data_X và target
data_X = df.drop(['Drug'], axis = 1).values
data_y = df['Drug']
print(data_X)
print(data_y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size = 0.2, random_state = 0)
print(X_train)
print(X_test)
print(y_train)
print(y_test)
print(len(X_train), len(X_test), len(y_train), len(y_test))
print(type(y_train))
test_pred = pred_test(6, X_train, X_test, y_train)
df_test_pred = pd.DataFrame(test_pred).drop([0], axis = 1)
df_test_pred.index = range(1, len(test_pred) + 1)
df_test_pred.columns = ['Predict']
df_actual = pd.DataFrame(y_test)
df_actual.index = range(1, len(y_test) + 1)
df_actual.columns = ['Actual']
pd.concat([df_test_pred, df_actual], axis = 1)

[[3 3 1 ... 0 1 0]
 [3 3 0 ... 0 1 0]
 [3 3 0 ... 0 1 0]
 ...
 [3 3 0 ... 1 1 0]
 [3 3 0 ... 1 0 1]
 [3 3 1 ... 0 0 1]]
0      DrugY
1      drugC
2      drugC
3      drugX
4      DrugY
       ...  
195    drugC
196    drugC
197    drugX
198    drugX
199    drugX
Name: Drug, Length: 200, dtype: object
[[3 3 1 ... 0 1 0]
 [3 3 0 ... 0 1 0]
 [3 3 0 ... 0 1 0]
 ...
 [3 3 1 ... 1 1 0]
 [3 3 0 ... 0 1 0]
 [3 3 1 ... 1 0 1]]
[[3 3 0 1 0 1 0 1 0]
 [3 3 1 0 0 0 1 1 0]
 [3 3 0 1 0 1 0 1 0]
 [3 3 0 1 1 0 0 0 1]
 [3 3 0 1 0 0 1 1 0]
 [3 3 1 0 0 1 0 0 1]
 [3 3 1 0 0 0 1 1 0]
 [3 3 1 0 0 1 0 0 1]
 [3 3 0 1 0 1 0 1 0]
 [3 3 0 1 0 0 1 0 1]
 [3 3 0 1 1 0 0 0 1]
 [3 3 1 0 1 0 0 0 1]
 [3 3 1 0 1 0 0 0 1]
 [3 3 0 1 0 1 0 0 1]
 [3 3 0 1 1 0 0 1 0]
 [3 3 0 1 0 1 0 1 0]
 [3 3 1 0 1 0 0 0 1]
 [3 3 1 0 0 0 1 1 0]
 [3 3 0 1 0 1 0 0 1]
 [3 3 0 1 1 0 0 0 1]
 [3 3 1 0 1 0 0 0 1]
 [3 3 0 1 0 0 1 1 0]
 [3 3 1 0 0 0 1 0 1]
 [3 3 1 0 0 1 0 0 1]
 [3 3 1 0 0 1 0 0 1]
 [3 3 1 0 0 0 1 1 0]
 [3 3 1 0 0 0 1 1 0]
 [3 3 1 0 0

,Predict,Actual
1,drugC,drugC
2,drugX,drugX
3,drugC,DrugY
4,drugB,DrugY
5,DrugY,DrugY
6,drugX,drugX
7,drugX,drugX
8,drugX,drugX
9,drugC,DrugY
10,drugX,drugX
